In [ ]:
# Version 1 - modified from Patrick's

In [23]:
import os, sys

sys.path.append('.')

from langchain.llms import Ollama

model="llama2:70b"
client = Ollama(base_url="http://localhost:11434", model=model)

In [24]:
# pip3 install gradio pypdf chromadb
# ollama pull nomic-embed-text

In [25]:
import gradio as gr
#import ollama
#from bs4 import BeautifulSoup as bs
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import OllamaEmbeddings
'''
# Load the data from the web URL
url = 'https://en.wikipedia.org/wiki/Ohiya'
loader = WebBaseLoader(url)
docs = loader.load()
'''
from langchain_community.document_loaders import PyPDFLoader
file_path = './50_Years_of_AI.pdf'
loader = PyPDFLoader(file_path)
docs = []
async for page in loader.alazy_load():
    docs.append(page)


In [26]:
# Split the loaded documents into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

In [27]:
# Create Ollama embeddings and vector store
embeddings = OllamaEmbeddings(model="nomic-embed-text")
vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings)


In [30]:
# Define the function to call the Ollama Llama3 model
def ollama_llm(question, context):
    formatted_prompt = f"Question: {question}\n\nContext: {context}"
#     response = client.chat.completions.create(model=model, messages=[{'role': 'system', 'content': '總是用繁體中文回答！'}, {'role': 'user', 'content': formatted_prompt}])
    response = Ollama.chat(
        model=model,  # Specify the model name
        messages=[
            {'role': 'system', 'content': '總是用繁體中文回答！'},
            {'role': 'user', 'content': formatted_prompt}
        ]
    )
#    return response.choices[0].message.content
    return response['content']


# Define the RAG setup
retriever = vectorstore.as_retriever()

def rag_chain(question):
    retrieved_docs = retriever.invoke(question)
    formatted_context = "\n\n".join(doc.page_content for doc in retrieved_docs)
    return ollama_llm(question, formatted_context)

# Define the Gradio interface
def get_important_facts(question):
    return rag_chain(question)

# Create a Gradio app interface
iface = gr.Interface(
  fn=get_important_facts,
  inputs=gr.Textbox(lines=2, placeholder="Please summarize in 500 words"),
  outputs="text",
  title="RAG with Llama3",
  description="Ask questions about the proveded context",
)

# Launch the Gradio app
iface.launch()


* Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.


In [31]:
import chromadb
print("chromadb installed and loaded successfully")


chromadb installed and loaded successfully


In [32]:
# Version 1 - same as above, but all codes in one cell

In [33]:
import os, sys

sys.path.append('.')

from langchain.llms import Ollama

model="llama2:70b"
client = Ollama(base_url="http://localhost:11434", model=model)

import gradio as gr
#import ollama
#from bs4 import BeautifulSoup as bs
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import OllamaEmbeddings
'''
# Load the data from the web URL
url = 'https://en.wikipedia.org/wiki/Ohiya'
loader = WebBaseLoader(url)
docs = loader.load()
'''
from langchain_community.document_loaders import PyPDFLoader
file_path = './50_Years_of_AI.pdf'
loader = PyPDFLoader(file_path)
docs = []
async for page in loader.alazy_load():
    docs.append(page)

# Split the loaded documents into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

# Create Ollama embeddings and vector store
embeddings = OllamaEmbeddings(model="nomic-embed-text")
vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings)


# Define the function to call the Ollama Llama3 model
def ollama_llm(question, context):
    formatted_prompt = f"Question: {question}\n\nContext: {context}"
#     response = client.chat.completions.create(model=model, messages=[{'role': 'system', 'content': '總是用繁體中文回答！'}, {'role': 'user', 'content': formatted_prompt}])
    response = Ollama.chat(
        model=model,  # Specify the model name
        messages=[
            {'role': 'system', 'content': '總是用繁體中文回答！'},
            {'role': 'user', 'content': formatted_prompt}
        ]
    )
#    return response.choices[0].message.content
    return response['content']


# Define the RAG setup
retriever = vectorstore.as_retriever()

def rag_chain(question):
    retrieved_docs = retriever.invoke(question)
    formatted_context = "\n\n".join(doc.page_content for doc in retrieved_docs)
    return ollama_llm(question, formatted_context)

# Define the Gradio interface
def get_important_facts(question):
    return rag_chain(question)

# Create a Gradio app interface
iface = gr.Interface(
  fn=get_important_facts,
  inputs=gr.Textbox(lines=2, placeholder="Please summarize in 500 words"),
  outputs="text",
  title="RAG with Llama3",
  description="Ask questions about the proveded context",
)

# Launch the Gradio app
iface.launch()


* Running on local URL:  http://127.0.0.1:7863

To create a public link, set `share=True` in `launch()`.


In [34]:
# Version 2 - modified by ChatGPT

# pip3 install langchain_ollama

In [36]:
import os, sys

sys.path.append('.')

from langchain_ollama import OllamaLLM
import gradio as gr
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import OllamaEmbeddings

# Load the data from a PDF
file_path = './50_Years_of_AI.pdf'
loader = PyPDFLoader(file_path)
docs = loader.load()

# Split the loaded documents into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

# Create Ollama embeddings and vector store
embeddings = OllamaEmbeddings(model="nomic-embed-text")
vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings)

# Define the function to call the Ollama Llama2 model
def ollama_llm(question, context):
    formatted_prompt = f"Question: {question}\n\nContext: {context}"
    llm = OllamaLLM(model="llama2:70b", base_url="http://localhost:11434")
    
    try:
        response = llm.generate(
            messages=[
                {'role': 'system', 'content': '總是用繁體中文回答！'},
                {'role': 'user', 'content': formatted_prompt}
            ]
        )
        return response['content']
    except Exception as e:
        return f"An error occurred: {str(e)}"

# Define the RAG setup
retriever = vectorstore.as_retriever()

def rag_chain(question):
    retrieved_docs = retriever.invoke(question)
    formatted_context = "\n\n".join(doc.page_content for doc in retrieved_docs)
    return ollama_llm(question, formatted_context)

# Define the Gradio interface
def get_important_facts(question):
    return rag_chain(question)

# Create a Gradio app interface
iface = gr.Interface(
  fn=get_important_facts,
  inputs=gr.Textbox(lines=2, placeholder="Please summarize in 500 words"),
  outputs="text",
  title="RAG with Llama3",
  description="Ask questions about the provided context",
)

# Launch the Gradio app
iface.launch()


* Running on local URL:  http://127.0.0.1:7864

To create a public link, set `share=True` in `launch()`.


In [ ]:
# version 3 - further modification from ChatGPT

In [39]:
import os, sys
sys.path.append('.')

from langchain_ollama import OllamaLLM
import gradio as gr
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import OllamaEmbeddings

# Load the data from a PDF
file_path = './linux-0.11_source.pdf'
loader = PyPDFLoader(file_path)
docs = loader.load()

# Split the loaded documents into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

# Create Ollama embeddings and vector store
embeddings = OllamaEmbeddings(model="nomic-embed-text")
vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings)

# Use the updated ollama_llm function
def ollama_llm(question, context):
    formatted_prompt = f"總是用繁體中文回答！\n\nQuestion: {question}\n\nContext: {context}"
    llm = OllamaLLM(model="llama3.1:70b", base_url="http://localhost:11434")
    
    try:
        response = llm.generate(prompts=[formatted_prompt])
        return response.generations[0][0].text
    except Exception as e:
        return f"An error occurred: {str(e)}"

# Define the RAG setup
retriever = vectorstore.as_retriever()

def rag_chain(question):
    retrieved_docs = retriever.invoke(question)
    formatted_context = "\n\n".join(doc.page_content for doc in retrieved_docs)
    return ollama_llm(question, formatted_context)

# Define the Gradio interface
def get_important_facts(question):
    return rag_chain(question)

# Create a Gradio app interface
iface = gr.Interface(
  fn=get_important_facts,
  inputs=gr.Textbox(lines=2, placeholder="Please summarize in 500 words"),
  outputs="text",
  title="RAG with Llama3",
  description="Ask questions about the provided context",
)

# Launch the Gradio app
iface.launch()


* Running on local URL:  http://127.0.0.1:7867

To create a public link, set `share=True` in `launch()`.


In [ ]:
# version 4- 讓 RAG 遺忘地一份文件

In [42]:
import os, sys
sys.path.append('.')

from langchain_ollama import OllamaLLM
import gradio as gr
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import OllamaEmbeddings

# Load the data from a PDF
file_path = './linux-0.11_source.pdf'
loader = PyPDFLoader(file_path)
docs = loader.load()

# Split the loaded documents into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

# Create Ollama embeddings and vector store
embeddings = OllamaEmbeddings(model="nomic-embed-text")

vectorstore = Chroma(embedding_function=embeddings, persist_directory='./chroma_db_linux011')
vectorstore.add_documents(documents=splits)

# Use the updated ollama_llm function
def ollama_llm(question, context):
    formatted_prompt = f"總是用繁體中文回答！\n\nQuestion: {question}\n\nContext: {context}"
    llm = OllamaLLM(model="llama3.1:70b", base_url="http://localhost:11434")
    
    try:
        response = llm.generate(prompts=[formatted_prompt])
        return response.generations[0][0].text
    except Exception as e:
        return f"An error occurred: {str(e)}"

# Define the RAG setup
retriever = vectorstore.as_retriever()

def rag_chain(question):
    retrieved_docs = retriever.invoke(question)
    formatted_context = "\n\n".join(doc.page_content for doc in retrieved_docs)
    return ollama_llm(question, formatted_context)

# Define the Gradio interface
def get_important_facts(question):
    return rag_chain(question)

# Create a Gradio app interface
iface = gr.Interface(
  fn=get_important_facts,
  inputs=gr.Textbox(lines=2, placeholder="Please summarize in 500 words"),
  outputs="text",
  title="RAG with Llama3",
  description="Ask questions about the provided context",
)

# Launch the Gradio app
iface.launch()


* Running on local URL:  http://127.0.0.1:7868

To create a public link, set `share=True` in `launch()`.


In [ ]:
# version 5 - 僅用 gradio當介面, 不用 RAG

In [1]:
import os, sys
sys.path.append('.')

from langchain_ollama import OllamaLLM
import gradio as gr

# 只使用 Ollama 模型進行問答
def ollama_llm(question):
    formatted_prompt = f"總是用繁體中文回答！\n\nQuestion: {question}"
    llm = OllamaLLM(model="llama3.1:70b", base_url="http://localhost:11434")
    
    try:
        response = llm.generate(prompts=[formatted_prompt])
        return response.generations[0][0].text
    except Exception as e:
        return f"An error occurred: {str(e)}"

# 定義 Gradio 介面
def get_important_facts(question):
    return ollama_llm(question)

# 創建 Gradio 應用介面
iface = gr.Interface(
  fn=get_important_facts,
  inputs=gr.Textbox(lines=2, placeholder="請輸入您的問題"),
  outputs="text",
  title="Ollama Chat",
  description="使用 Llama3 模型直接回答您的問題",
)

# 啟動 Gradio 應用
iface.launch()

/home/kpl/miniconda3/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


In [ ]:
# version 6 - 修改 gradio介面

In [2]:
import os, sys
sys.path.append('.')

from langchain_ollama import OllamaLLM
import gradio as gr

# 初始化一個全局的問答歷史列表
chat_history = []

# Ollama 聊天模型函數
def ollama_llm(question):
    formatted_prompt = f"總是用繁體中文回答！\n\nQuestion: {question}"
    llm = OllamaLLM(model="llama3.1:70b", base_url="http://localhost:11434")
    
    try:
        response = llm.generate(prompts=[formatted_prompt])
        return response.generations[0][0].text
    except Exception as e:
        return f"An error occurred: {str(e)}"

# Gradio 的主邏輯函數，用於處理問題並更新聊天歷史
def get_important_facts(question):
    # 獲取模型回答
    answer = ollama_llm(question)
    
    # 更新聊天歷史
    chat_history.append((question, answer))
    
    # 將所有聊天歷史格式化成單一字符串，顯示於輸出
    chat_output = ""
    for q, a in chat_history:
        chat_output += f"**問題**: {q}\n\n**回答**: {a}\n\n"
    
    return chat_output

# 創建 Gradio 應用介面
iface = gr.Interface(
  fn=get_important_facts,
  inputs=gr.Textbox(lines=2, placeholder="請輸入您的問題"),
  outputs="markdown",  # 使用 markdown 格式顯示聊天歷史
  title="Ollama Chat",
  description="使用 Llama3 模型進行問答，顯示完整對話歷史",
)

# 啟動 Gradio 應用
iface.launch()


* Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.
